## Churn Prediction

#### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### Load DataSet

In [2]:
df = pd.read_csv('/home/rguktrkvalley/Desktop/churn_prediction/Churn_Modelling.csv')
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

### Encoding the categorical Labels

In [3]:
from sklearn.preprocessing import LabelEncoder
LeX=LabelEncoder()
X[:,2]=LeX.fit_transform(X[:,2])

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder='passthrough')
X=np.array(ct.fit_transform(X))

#### Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

#### Spliting the dataset into train and test data

In [6]:
#Splitting dataset into train and test set
from sklearn.model_selection import train_test_split
#Set any int as random state for reprodicibility
X_train, X_test, Y_train, Y_test=train_test_split(X,y,test_size=0.2,random_state=42)


#### Import model and layers

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2025-08-18 05:13:23.982780: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 05:13:26.616362: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 05:13:26.620435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-18 05:13:32.868375: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
input_dim=X_train.shape[1]

#### Inputing layers into model

In [9]:
model=Sequential([Dense(64,activation='relu',input_shape=(input_dim,)),    #Adding first hidden layer
                  Dense(32,activation='relu'),                            #Second hidden layer
                  Dense(1,activation='sigmoid')]) 

#### Compiling the model

In [10]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Training the model

In [11]:
history=model.fit(X_train,Y_train,batch_size=32, epochs=55,validation_split=0.2)

Epoch 1/55
200/200 [==============================] - 2s 3ms/step - loss: 0.4725 - accuracy: 0.7920 - val_loss: 0.4091 - val_accuracy: 0.8288
Epoch 2/55
200/200 [==============================] - 0s 2ms/step - loss: 0.3954 - accuracy: 0.8397 - val_loss: 0.3742 - val_accuracy: 0.8481
Epoch 3/55
200/200 [==============================] - 0s 2ms/step - loss: 0.3614 - accuracy: 0.8537 - val_loss: 0.3593 - val_accuracy: 0.8494
Epoch 4/55
200/200 [==============================] - 0s 2ms/step - loss: 0.3463 - accuracy: 0.8589 - val_loss: 0.3612 - val_accuracy: 0.8519
Epoch 5/55
200/200 [==============================] - 0s 2ms/step - loss: 0.3403 - accuracy: 0.8589 - val_loss: 0.3510 - val_accuracy: 0.8462
Epoch 6/55
200/200 [==============================] - 0s 2ms/step - loss: 0.3373 - accuracy: 0.8614 - val_loss: 0.3503 - val_accuracy: 0.8581
Epoch 7/55
200/200 [==============================] - 0s 2ms/step - loss: 0.3329 - accuracy: 0.8616 - val_loss: 0.3472 - val_accuracy: 0.8506
Epoch 

#### Evaluating The model

In [12]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

63/63 [==============================] - 0s 2ms/step - loss: 0.3588 - accuracy: 0.8565
Test Loss: 0.3588
Accuracy: 0.8565


#### Single Prediction and test data prediction

In [13]:
print(model.predict(sc.transform([[1.0, 0.0, 0.0, 1, 447, 31, 7, 0.0, 4, 1,1, 519360]]))>0.5)
print(model.predict(sc.transform([[0.0,1.0,0.0, 0, 315, 25, 3, 12344, 2, 1,0, 12425]]))>0.5)
predictions=model.predict(X_test)

1/1 [==============================] - 0s 86ms/step
[[False]]
1/1 [==============================] - 0s 21ms/step
[[False]]
63/63 [==============================] - 0s 1ms/step


In [14]:
Y_pred=model.predict(X_test)
Y_pred=(Y_pred>0.5)
print(np.concatenate((Y_pred.reshape(len(Y_pred),1),Y_test.reshape(len(Y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [1 1]
 [0 1]
 [0 1]]


#### Confusion metrix and accuracy

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(Y_test,Y_pred)
print(cm)
accuracy_score(Y_test,Y_pred)

[[1513   94]
 [ 193  200]]


0.8565

#### Saving The model

In [16]:

# AFTER training is completed (after model.fit)
from tensorflow.keras.models import load_model
import joblib

# ✅ Save the model
model.save("model.keras")  # or "model.h5"
model.save("model.h5")
# ✅ Save the scaler
joblib.dump(sc, "scaler.pkl")


/home/rguktrkvalley/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['scaler.pkl']

In [19]:
# Import libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Scale features (optional but good if features have very different scales)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, Y_train)

# Predictions
y_pred = rf.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(Y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(Y_test, y_pred))
print("\nClassification Report:\n", classification_report(Y_test, y_pred))


Accuracy: 0.865

Confusion Matrix:
 [[1543   64]
 [ 206  187]]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.75      0.48      0.58       393

    accuracy                           0.86      2000
   macro avg       0.81      0.72      0.75      2000
weighted avg       0.86      0.86      0.85      2000

